In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv(r'stpa-step1-dataset.csv')
display(df)
print(df['label'].value_counts())

,req,label
0,Humans and/or human assets on earth are killed...,loss
1,Humans and/or human assets off of the earth ar...,loss
2,Organisms on any of the moons of the outer pla...,loss
3,The scientific data corresponding to the missi...,loss
4,The scientific data is rendered unusable (e.g....,loss
...,...,...
1073,The Laboratory Supervisors must be a part of t...,constraint
1074,Operators must conclude operational laboratory...,constraint
1075,Operational laboratory trainings must approach...,constraint
1076,The Laboratory Managers must periodically impl...,constraint


label
hazard          408
constraint      316
loss            254
exloss           47
exhazard         34
exconstraint     19
Name: count, dtype: int64


In [3]:
experiment = '1' # @param ['1', '2']

In [4]:
if(experiment=='1'):
  df = df[~df['label'].isin(['exconstraint', 'exloss', 'exhazard'])]

elif(experiment=='2'):
  df['label'] =  df['label'].replace(['exconstraint'], 'constraint')
  df['label'] =  df['label'].replace(['exloss'], 'loss')
  df['label'] =  df['label'].replace(['exhazard'], 'hazard')

print(df['label'].value_counts())

y = df['label'].to_list()
x = df['req'].to_list()
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0, test_size=0.2, stratify=y)

print(len(x_train), len(x_test))

label
hazard        408
constraint    316
loss          254
Name: count, dtype: int64
782 196


In [5]:
def print_results(name, label, prediction):
  print(name)
  print('Acc:       ', accuracy_score(label, prediction))
  print('precision: ', precision_score(label, prediction, average='macro'))
  print('recall:    ', recall_score(label, prediction, average='macro'))
  print('F1:        ', f1_score(label, prediction, average='macro'))

In [6]:
pipe_mnb = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('nb', MultinomialNB(alpha=1)),
])

param_mnb = {
    'nb__alpha': [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
}

gsearch_mnb = GridSearchCV(pipe_mnb, param_mnb, scoring='accuracy', cv=5, refit=True)
gsearch_mnb.fit(x_train, y_train)
print(gsearch_mnb.best_params_)

{'nb__alpha': 1}


In [ ]:
pipe_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('svm', SVC(random_state=0))
])

param_svm = {
    'svm__gamma': [0.0001, 0.001, 0.1, 1, 10],
    'svm__C': [0.0001, 0.001, 0.1, 1, 10],
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

gsearch_svm = GridSearchCV(pipe_svm, param_svm, scoring='accuracy', cv=5, refit=True)
gsearch_svm.fit(x_train, y_train)
print(gsearch_svm.best_params_)

In [ ]:
#Test classifiers

pipe_mnb_predicted_test = gsearch_mnb.predict(x_test)
print_results('MultinomialNB', y_test, pipe_mnb_predicted_test)

pipe_svm_predicted_test = gsearch_svm.predict(x_test)
print_results('SVM', y_test, pipe_svm_predicted_test)